## Explore metadata for a curated dataset

This notebook shows how to connect to a curated TDM dataset and explore the metadata.

In [ ]:
import pandas as pd

from tdm_client import Dataset

Initialize a TDM dataset object with the dataset ID provided by the Digital Scholars Workbench or in the email you received after curating your dataset.

In [ ]:
dset = Dataset('a517ef1f-0794-48e4-bea1-ac4fb8b312b4')

Find total number of documents in the dataset. 

In [ ]:
len(dset)

After initializing the `Dataset`, the documents in the dataset are downloaded in the background.

A list of all of the items in the Dataset is available.

In [ ]:
dset.items[:5]

The document metadata can be retrieved by calling the `get_metadata` method. The metadata is a list of Python dictionaries containing attributes for each document.

In [ ]:
metadata = dset.get_metadata()

Print the metadata for the first document in the dataset.

In [ ]:
print(metadata[0])

Convert the metadata to a Pandas dataframe to take advantage of its plotting and manipulation functions.

In [ ]:
df = pd.DataFrame(metadata)

Print the first 10 rows of the dataframe

In [ ]:
df.head()

Find the year range of documents in this dataset

In [ ]:
print('{} to {}'.format(df['yearPublished'].min(), df['yearPublished'].max()))

Now do some preliminary analysis. Let's say you want to plot the number of volumes by decade in the sample set. 

Since `decade` isn't a value in our dataset, we need to add it to the dataframe. This can be done with the following step.

In [ ]:
df['decade'] = df['yearPublished'] - ( df['yearPublished'] % 10)

Print the first 10 rows of the dataframe again to see how the column was applied.

In [ ]:
df.head()

Now use the built in plotting tools of Pandas to plot the number of issues from each provider by decade.

In [ ]:
df.groupby(['decade', 'provider'])['id'].agg('count').unstack()\
    .plot.bar(title='Issues by decade', figsize=(20, 5), fontsize=12, stacked=True);

And do the same for the total number of pages.

In [ ]:
df.groupby(['decade', 'provider'])['pageCount'].agg('sum').unstack()\
    .plot.bar(title='Pages by decade', figsize=(20, 5), fontsize=12, stacked=True);